# Feature engineering

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [2]:
# parameters
data_path = 'data'
data_set = 'clean_document.csv'

In [3]:
complete_corpus = pd.read_csv('{}/{}'.format(data_path, data_set))
complete_corpus.head()

,class,processed_text
0,0,wto chief debat trump ralli support trade head...
1,0,catalonia find friend among eu leader european...
2,1,shock fed govern grant disabl statu benefit sp...
3,1,climat scammer al gore utterli embarrass expla...
4,1,lol leftist ca congresswoman tonight debat deb...


In [4]:
def count_features(values, keys):
    summed_values = values.sum(axis=0)
    zipped_dictionary = zip(summed_values, keys)
    sorted_zipped_dictionary = sorted(zipped_dictionary, reverse=True)
    return sorted_zipped_dictionary

In [5]:
def calculate_term_frequency(X):
    term_frequency_array = np.zeros(shape=X.shape)
    iteration = 0
    for array in X:
        if array.sum() > 0:
            term_frequency_array[iteration] = array / array.sum()
        else:
            term_frequency_array[iteration] = array
        iteration += 1
    return term_frequency_array        

In [6]:
def test_calculate_term_frequency():
    X = np.array([[4, 2, 7, 4, 9, 1, 0], 
                  [6, 3, 2, 0, 0, 8, 1]])
    Y = calculate_term_frequency(X)
    Y_result = np.array([[4/27, 2/27, 7/27, 4/27, 9/27, 1/27, 0/27], 
                         [6/20, 3/20, 2/20, 0/20, 0/20, 8/20, 1/20]])
    comparison = Y == Y_result
    equal_arrays = comparison.all()
    assert equal_arrays == True, 'test_calculate_term_frequency: check of test failed'
    print('all test_calculate_term_frequency tests passed')

In [7]:
test_calculate_term_frequency()

all test_calculate_term_frequency tests passed


In [8]:
def calculate_term_frequency_inverse_document_frequency(X):
    """
    Calculate inverse term frequency inverse document frequency (TF-IDF).
    """
    inverse_document_frequency = np.zeros(shape=X.shape)
    term_frequency = calculate_term_frequency(X)
    corpus_sum = term_frequency # matrix term_frequency is used later, therefore create a copy
    
    # count an n-gram only once per document
    corpus_sum[corpus_sum > 0] = 1
    
    # inverse document frequency (idf) is the number of documents in the corpus (here X.shape[0]), 
    # divided by the number of documents where a word appears
    idf_single_array = 1 + np.log(X.shape[0] / corpus_sum.sum(axis=0)) # sum over all documents

    # for matrix matrix multiplication between term_freq and term_freq_inv_doc_freq elementwise
    iteration = 0
    for array in X:
        inverse_document_frequency[iteration] = idf_single_array
        iteration += 1
        
    term_frequency_inverse_document_frequency = np.multiply(term_frequency, inverse_document_frequency)
    return term_frequency_inverse_document_frequency

In [9]:
def build_vocabulary(df, feature_size, n_gram_size):
    corpus = df['processed_text'].values
    vectorizer = CountVectorizer(max_features=feature_size, ngram_range=(n_gram_size, n_gram_size))
    feature_matrix = vectorizer.fit_transform(corpus)
    feature_vocabulary = vectorizer.get_feature_names() # feature order
    return feature_vocabulary, feature_matrix.toarray()

In [10]:
def build_feature(df, feature_size, n_gram_size):
    """
    Build the features necessary for machine learning. 
    
    Args:
    df: (dataframe) Corpus with processed texts saved as feature 'processed_text' and its corresponding class 
    available as 'class'.
    feature_size: (int) Defines the number of top n-grams extracted from the corpus.
    n_gram_size: (int) Defines the n-gram size (contiguous sequence of items with length n).
    
    Returns:
    feature_vocabulary: (list of n-grams) The vocabulary with the top feature_size n-grams.
    feature_occurance: (numpy array) The occurance matrix of n-grams, number of rows = feature_size, 
    number of columns = corpus length.
    feature_statistics: (dictionary) Dictionary with size feature_size sorted according to top n-gram.
    term_freq: (numpy array) Term frequency matrix
    term_freq_inverse_doc_freq: (numpy array) Maxrix according to term frequency inverse document frequency model.
    """
    feature_vocabulary, feature_occurance = build_vocabulary(complete_corpus, feature_size, n_gram_size)
    feature_statistics = count_features(feature_occurance, feature_vocabulary)
    term_freq = calculate_term_frequency(feature_occurance)
    term_freq_inverse_doc_freq = calculate_term_frequency_inverse_document_frequency(feature_occurance)
    return feature_vocabulary, feature_occurance, feature_statistics, term_freq, term_freq_inverse_doc_freq

# Test case

Define a test case with 10 most used words in the corups and a n-gram of 1.

In [11]:
vocabulary, occurance, statistics, term_freq, term_freq_inverse_doc_freq = build_feature(complete_corpus, 10, 1)

In [12]:
statistics

[(649, 'trump'),
 (586, 'said'),
 (272, 'would'),
 (270, 'state'),
 (221, 'presid'),
 (202, 'year'),
 (193, 'clinton'),
 (191, 'peopl'),
 (185, 'one'),
 (174, 'say')]

# Create feature datasets

In [26]:
def data_split_and_save(df, tf, tf_idf, number_features, n_gram_size):
    
    data_list = ['tf', 'tf-idf']
    
    for feature in data_list:
        print('split and save {}...'.format(feature), end='')
        
        if feature == 'tf':
            X = tf # numpy
        else:
            X = tf_idf # numpy
    
        y = df['class'].values # numpy

        # first split 60% as training data, rest 40% as test which will be split later 20% test, 20% validation
        X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=0.60, random_state=1)

        # fifity / fifity split validation and test data
        X_test, X_validation, y_test, y_validation = train_test_split(X_rest, y_rest, train_size=0.50, random_state=1)

        data_path_train = 'train-{}-{}-{}'.format(feature, number_features, n_gram_size) 
        data_path_test = 'test-{}-{}-{}'.format(feature, number_features, n_gram_size) 
        data_path_validation = 'validation-{}-{}-{}'.format(feature, number_features, n_gram_size) 
        data_path_list = [data_path_train, data_path_test, data_path_validation]

        for check_path in data_path_list:
            if not os.path.exists(check_path):
                os.mkdir(check_path)

        train_df = pd.concat([pd.DataFrame(y_train), pd.DataFrame(X_train)], axis=1)
        test_df = pd.concat([pd.DataFrame(y_test), pd.DataFrame(X_test)], axis=1)
        validation_df = pd.concat([pd.DataFrame(y_validation), pd.DataFrame(X_validation)], axis=1)

        train_df.to_csv('{}/{}'.format(data_path_train, 'train.csv'), index=False, header=False)
        test_df.to_csv('{}/{}'.format(data_path_test, 'test.csv'), index=False, header=False)
        validation_df.to_csv('{}/{}'.format(data_path_validation, 'validation.csv'), index=False, header=False)
        
        print('done')
    

In [29]:
def create_feature_data(df, number_features, n_gram_size):
    voca, matrix, stat, tf, tf_idf = build_feature(complete_corpus, number_features, n_gram_size)
    data_split_and_save(df, tf, tf_idf, number_features, n_gram_size)

In [30]:
create_feature_data(complete_corpus, 10, 1)

split and save tf...done
split and save tf-idf...done
